Metoda potęgowa jest najprostszą metodą pozwalającą na znalezienie największej (co do modułu) wartości własnej dla macierzy symetrycznej (czyli takiej, która posiada jedynie wartości własne będące liczbami rzeczywistymi).

Zasada działania jest prosta: przekształcając dowolny, **niezerowy** wektor przez macierz ulegnie on transformacji. Składowe tego wektora na wektory własne zostaną rozciągnięte (lub skrócone) zgodnie ze skojarzonymi z tymi wektorami wartościami własnymi. Oznacza to jeżeli nasz wektor startowy jest wektorem własnym to nie ulegnie obrotowi (może zostać ewentualnie wydłużony lub skrócony).

Jeżeli nasz wektor startowy **nie jest** wektorem własnym, to zostanie on obracany w kierunku wektora własnego skojarzonego z największą (co do modułu) wartością własną.

In [ ]:
using LinearAlgebra

# Iloraz Rayleigha pozwala na znalezienie wartości własnej
# macierzy A, skojarzonej z wektorem własnym v
function rayleighquotient(v, A)
    (v' * A * v) / (v' * v)
end

W metodzie potęgowej operujemy na macierzy i wektorze. Możemy "odzyskać" skojarzoną z wektorem wartość własną przy pomocy ilorazu Rayleigha:

$$\lambda = \frac{v^T\cdot \mathbf{A}\cdot v}{v^T \cdot v}$$

In [ ]:
function poweriteration(A)
    maxiter = 1000
    λ = NaN
    n = size(A, 1)
    vi = randn(n)
    vj = similar(vi)
    for i=1:maxiter
        vj .= A * vi
        normalize!(vj)
        vi, vj = vj, vi
        λ = rayleighquotient(vi, A)
    end
    return λ, vi
end

Powyższa implementacja jest jeszcze do delikatnego poprawienia pod kątem wydajności (tak samo jak iloraz Rayleigha).

In [ ]:
A = randn(5,5); A *= A'
λ, v = poweriteration(A)
@show λ;
@show v;

In [ ]:
values, vectors = eigen(A)

In [ ]:
using BenchmarkTools
@btime poweriteration($A);
@btime rayleighquotient($v, $A);

In [ ]:
function rayleighquotient(v, A)
    n = length(v)
    R = 0.0
    for i=1:n, j=1:n
        R += v[i] * A[i,j] * v[j]
    end
    return R
end

function poweriteration(A)
    maxiter = 1000
    λ = NaN
    n = size(A, 1)
    vi = randn(n)
    vj = similar(vi)
    for i=1:maxiter
        mul!(vj, A, vi)
        normalize!(vj)
        vi, vj = vj, vi
        λ = rayleighquotient(vi, A)
    end
    return λ, vi
end

@btime poweriteration($A);
@btime rayleighquotient($v, $A);

Czy można wykorzystać metodę potęgową do znalezienia **wszystkich** wartości własnych?
Tak, przy pomocy deflacji. Deflacja pozwala na wyzerowanie z macierzy wybranej wartości własnej.

$$\mathbf{B} = \mathbf{A} - \lambda_i \cdot v_i \cdot v^{T}_i$$

In [ ]:
A = rand(5,5); A = A' * A;
values, vectors = eigen(A)

In [ ]:
B = A .- values[3] * vectors[:,3] * vectors[:,3]'
eigen(B)

In [ ]:
function wlasne(A)
    n = size(A, 1)
    B = copy(A)
    λ = zeros(n)
    for i=1:n
        lambda, vector = poweriteration(B)
        λ[i] = lambda
        B .-= lambda * vector * vector'
    end
    return sort!(λ)
end

λ, _ = poweriteration(A)
λs = wlasne(A)
values, _ = eigen(A)
using PyPlot
figure(dpi=150)
scatter(real.(λs), imag.(λs), s=100)
scatter(real.(values), imag.(values))
xlabel("Re{λ}")
ylabel("Im{λ}")
legend(("Power Iteration", "Eigen"))
#display(gcf())

In [ ]:
A = diagm(rand(120))
λ = wlasne(A)
values, vectors = eigen(A)
figure(dpi=150)
plot(λ, "o")
plot(values, ".")
xlabel("numer wartości własnej (-)")
ylabel("wartość własna (-)")
legend(("Power Iteration", "Eigen"))
#display(gcf())

Pamiętając o właściwości, że wartości własne macierzy odwrotnej są odwrotnościami oryginalnej macierzy możemy
też zapisać wersję metody potęgowej znajdującej **najmniejszą** (co do modułu) wartość własną. Zamiast mnożenia wektora przez macierz będziemy mnożyć wektor przez odwrotność macierzy. 

Metoda potęgowa: $v_{i+1} = \mathbf{A} \cdot v_{i}$.

Odwrotna metoda potęgowa: $v_{i+1} = \mathbf{A}^{-1} \cdot v_{i}$.

Oczywiście, znajdowanie macierzy odwrotnej jest najczęściej bardzo pracochłonne, więc możemy spróbować innych metod na znalezienie $\mathbf{A} \cdot v_{i+1} = v_{i}$.
Warto zwrócić uwagę, że w naszym przypadku będziemy wykonywać wiele iteracji, ale macierzowy układ równań będzie taki sam. Na razie przyjmiemy "na wiarę", że istnieją metody pozwalające przyspieszyć rozwiązać tego typu układów równań dla różnych wektorów po prawej stronie równania (patrz wykorzystywana poniżej funkcja `factorize`).

In [ ]:
function inverseiteration(A)
    maxiter = 1000
    λ = NaN
    n = size(A, 1)
    I = diagm(ones(n))
    vi = randn(n)
    Ainv = factorize(A)
    
    normalize!(vi)
    for i=1:maxiter
        vi = Ainv \ vi
        normalize!(vi)
    end
    
    λ = rayleighquotient(vi, A)
    return λ, vi
end

In [ ]:
A = rand(10, 10); A = A' * A
eigvals(A)

Posiadając funkcje do znajdowania najmniejszej i największej wartości własnej możemy dowiedzieć się ciekawych rzeczy o naszym operatorze $\mathbf{A}$, np. o jego współczynniku uwarunkowania.
Współczynnik uwarunkowania mówi o tym jak błędy skojarzone z wektorem $x$ propagują się na wektor wyjściowy $y = \mathbf{A} \cdot x$.

Współczynnik uwarunkowania można policzyć jako iloraz największej wartości własnej do najmniejszej.

In [ ]:
function uwarunkowanie(A)
    λmin, _ = inverseiteration(A)
    λmax, _ = poweriteration(A)
    return λmax/λmin
end

@btime uwarunkowanie($A)

In [ ]:
cond(A) # condition number

Co więcej, odwrotna metoda potęgowa pozwala nam znaleźć **dowolną** wartość własną, jeżeli tylko domyślamy się jej wartości.
Wykorzystamy tu własność związaną z przesunięciem widma macierzy.

Macierz $\mathbf{A} - \mu \cdot \mathbf{I}$ posiada takie same wartości własne jak $\mathbf{A}$ ale przesunięte o $\mu$, tj. $\lambda_1 - \mu$, $\lambda_2 - \mu$, $\lambda_3 - \mu$, $\ldots, \lambda_N - \mu$.

In [ ]:
function inverseiteration(A, μ)
    N = size(A, 1)
    I = diagm(ones(N))
    λ, v = inverseiteration(A - μ * I)
    return λ + μ, v
end

In [ ]:
λ, _ = inverseiteration(A, 1.0)
@show λ;

Możemy przyspieszyć prędkość zbieżności odwrotnej metody potęgowej poprzez wielokrotne przesuwanie widma w pogoni za wartością własną najbliższą wartości $\mu$.

In [ ]:
function rayleighiteration(A, μ=0.0)
    maxiter = 1000
    n = size(A, 1)
    I = diagm(ones(n))
    vi = randn(n)
    normalize!(vi)
    for i=1:maxiter
        vi .= (A - μ * I) \ vi
        normalize!(vi)
        μ  = rayleighquotient(vi, A)
    end
    
    λ = rayleighquotient(vi, A)
    return λ, vi
end

In [ ]:
λ, _ = rayleighiteration(A, 2.086631270326873)
@show λ;

In [ ]:
eigvals(A)